## Mesonet API

Mesonet provide access to a network of weather stations.  For all but huge data volumes, the service is provided at no cost.

#### Configuration

Replace with your Mesonet API token, obtainable here:

https://developers.synopticdata.com

In [1]:
token = 'ff71330455da49468c2f7355070f2099'

In [2]:
timeseries_url = 'https://api.synopticdata.com/v2/stations/timeseries'

In [3]:
%%writefile branding_ecoacumen.json
{
"logo": "ecoacumen_logo.png",
"title": "EcoAccumen Weather Data",
"subtitle": "Continuous Environmental Monitoring"
}

Overwriting branding_ecoacumen.json


In [4]:
%%writefile branding_pge.json
{"logo": "pge_logo.png",
"title": "OmniSci Weather Data Loader",
"subtitle": "Continuous Powerline Asset Monitoring"
}

Overwriting branding_pge.json


#### Code

In [13]:
from __future__ import print_function
import ipywidgets as ux
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import interactive_output
import os
import requests
import pandas as pd
# for location lookups
import osmnx
import us
import json

In [6]:
%run ../omnigeo.ipynb

Using s2 library
Operating without GDAL library - functionality will be reduced
Using pymapd version 0.9.0
OmniSci connection established on python global variable "con"
Using CPU memory (/dev/shm) for temp geo files
Filesystem     1K-blocks  Used Available Use% Mounted on
tmpfs          115549360 82192 115467168   1% /dev/shm


In [7]:
# gets passed in two python datetimes
# or defaults to last month
def make_date_param(start=None, end=None):
    if start == None or end == None:
        # default to last month in hours
        return(f'&recent={24*30}')
    else:
        # hmm... actually no hour picker in ux and not so critical, so defaulting to midnight for now
        # :02 forces 0-padding of months and days required by API
        date_param = f'&start={start.year}{start.month:02}{start.day:02}1200' 
        date_param += f'&end={end.year}{end.month:02}{end.day:02}1200'
        return(date_param)

In [8]:
date_label = ux.Label("Specify a date range (defaults to last 30 days)")
#display(date_label)

In [9]:
start_date_picker = ux.DatePicker(description="Start Date:")
end_date_picker = ux.DatePicker(description="End Date:")

In [10]:
date_pickers = ux.HBox([start_date_picker, end_date_picker])
#display(date_label,date_pickers)

In [14]:
# get branding from config file saved above, and load into dictionary
with open("branding_pge.json") as fp:
    branding = json.load(fp)
branding

{'logo': 'pge_logo.png',
 'title': 'OmniSci Weather Data Loader',
 'subtitle': 'Continuous Powerline Asset Monitoring'}

In [15]:
def logo_widget(logo_file_path=''):
    with open("branding_pge.json") as fp:
        branding = json.load(fp)
        print('Using branding from file')
    if len(logo_file_path) == 0:
        logo_file_path = f"data/images/{branding['logo']}"
    if os.path.exists(logo_file_path):
        file = open(logo_file_path, "rb")
        image = file.read()
        logo_widget = ux.Image(
            value=image,
            format='png',
            width=100,
            height=100,
        )
        return(logo_widget)
    else:
        print(f"Logo {logo_file_path} not found")

In [16]:
logo = logo_widget()
if not branding == None:
    title = ux.HTML(f"<h2>{branding['title']}</h2>")
    subtitle = ux.HTML(f"<h4>{branding['subtitle']}</h4>")
else:
    title = ux.HTML("<h2>Weather Data</h2>")
    subtitle = ux.HTML("<h4>Continuous Environmental Monitoring</h4>")
if logo is not None:
    items = [logo, title, subtitle]
else:
    items = [title, subtitle]
    
header = ux.VBox(items)

Using branding from file


In [17]:
units_picker = ux.RadioButtons(
    options=['English', 'Metric'],
    description='Units:',
    disabled=False
)

In [18]:
def make_units_param(units_picker):
    try:
        u = f'&units={units_picker.value}'
        return(u)
    except:
        print('Problem reading units, defaulting')
        return('&units=Metric')

In [19]:
def update_url(start_date_picker, end_date_picker, units_picker):
    make_date_param(start_date_picker, end_date_picker)
    make_units_param(units_picker)
    print("URL updated with date and unit changes")

In [20]:
def make_states_menu():
    stateslist = []
    for s in us.STATES:
        stateslist.append(s.name)

    states_menu = ux.Dropdown(
        options=stateslist,
        value='California',
        description='State:',
        disabled=False,
    )
    return(states_menu)

In [21]:
states_menu = make_states_menu()

In [22]:
def make_counties_menu():
    # get a mapping between name and fips from us
    name_fips_dict = us.states.mapping('name', 'fips')
    st_fips = name_fips_dict[states_menu.value]
    # get counties from OmniSci and return as dataframe
    counties_table = 'us_counties_2017'
    q = f"SELECT name FROM {counties_table} WHERE STATEFP = '{st_fips}'"
    df = con.select_ipc(q)
    counties_list = list(df['name'])

    counties_menu = ux.Dropdown(
        options=counties_list,
        value='San Francisco',
        description='County:',
        disabled=False,
    )
    return(counties_menu)

In [24]:
counties_menu = make_counties_menu()
#display(make_counties_menu())

In [25]:
def location_spec():
    loc_label = ux.Label("Location")
    loc_text_control = ux.Text("Enter OSM placename here or pick from menus")
    return(ux.VBox([loc_label, loc_text_control, states_menu, counties_menu]))

#### Geocode from Arbitrary Text Entry

In [26]:
import geopy

In [27]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="ecoacumen_weather")
location = geolocator.geocode("Placer County, CA")
print(location.address)


Placer County, California, USA


In [28]:
print((location.latitude, location.longitude))

(39.1012064, -120.7650606)


In [29]:
print(location.raw)

{'place_id': 198845583, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 396511, 'boundingbox': ['38.7112111', '39.3164742', '-121.4844692', '-120.0024755'], 'lat': '39.1012064', 'lon': '-120.7650606', 'display_name': 'Placer County, California, USA', 'class': 'boundary', 'type': 'administrative', 'importance': 0.761293122268603, 'icon': 'https://nominatim.openstreetmap.org/images/mapicons/poi_boundary_administrative.p.20.png'}


In [30]:
def update_loc():
    print('not implemented yet')

In [31]:
out_loc = ux.interactive_output(update_loc, {'location_spec': location_spec })

AttributeError: 'function' object has no attribute 'observe'

In [32]:
build_button = ux.Button(
    description='Build',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to create custom weather service',
    icon='check'
)

In [33]:
cta = ux.HBox([build_button])

#### User Interface

In [34]:
out = ux.interactive_output(update_url, {'start_date_picker': start_date_picker, 'end_date_picker': end_date_picker, 'units_picker': units_picker})
display(header, date_label, date_pickers, units_picker, location_spec(), cta)

Label(value='Specify a date range (defaults to last 30 days)')

RadioButtons(description='Units:', options=('English', 'Metric'), value='English')

In [35]:
out

Output(outputs=({'name': 'stdout', 'text': 'Problem reading units, defaulting\nURL updated with date and unit …

Then what... upload to s3, to OmniSci, plot map here, display nice tabular dataframe inline?

#### Once Specification Done Above, can make final URL

In [37]:
def make_loc_params():
    county = counties_menu.value.replace(' ','%20')
    # actually...need state code given state name
    name_abr_dict = us.states.mapping('name', 'abbr')
    st_abbr = name_abr_dict[states_menu.value]
    return(f'&county={county}&state={st_abbr}')

In [38]:
def make_url():
    url = timeseries_url + '?&token=' + token 
    url += make_loc_params() 
    url += make_date_param(start_date_picker.value, end_date_picker.value)
    url += make_units_param(units_picker)
    url += '&vars=wind_speed,wind_direction'
    return(url)

In [39]:
url = make_url()
url

'https://api.synopticdata.com/v2/stations/timeseries?&token=ff71330455da49468c2f7355070f2099&county=San%20Francisco&state=CA&recent=720&units=English&vars=wind_speed,wind_direction'

In [43]:
def add_station_data(response, i):
    sdf = pd.DataFrame(columns=['station_id','station_name','obs_datetime','wind_speed','wind_speed_units','wind_direction','wkt'])
    print(f'Extracting data for station {i}')
    station_data = response.json()['STATION'][i]
    
    station_id = station_data['STID']
    sdf['station_id'] = station_id
    station_name = station_data['NAME']
    sdf['station_name'] = station_name
    lon = station_data['LONGITUDE']
    lat = station_data['LATITUDE']
    station_loc =  f"POINT({lon} {lat})"
    sdf['wkt'] = station_loc
    avail_sensors = list(station_data['SENSOR_VARIABLES'].keys())
    print(f'Working on station {i}: {station_name}')
    wsu = response.json()['UNITS']['wind_speed']
    sdf['wind_speed_units'] = wsu
    
    sdf.head()
    
    # this gets a dictionary with date_time keys
    station_obs = station_data['OBSERVATIONS']
    obs_df = pd.DataFrame(station_obs)
    obs_count = obs_df.size
    print(f'Found {obs_count} observations')

    if obs_count > 0:
        try:
            sdf['obs_datetime'] = obs_df['date_time']
        except:
            sdf['obs_datetime'] = None
            print(f'Unexpected error: could not find data/times for observations for station {i}')
        try:
            sdf['wind_speed'] = obs_df['wind_speed_set_1']
        except:
            sdf['wind_speed'] = None
            print(f'Wind speed data not available for station {i}, these sensors found: {avail_sensors}')
        try:
            sdf['wind_direction'] = obs_df['wind_direction_set_1']
        except:
            sdf['wind_direction'] = None
        sdf['station_id'] = station_id
        sdf['station_name'] = station_name
        sdf['wkt'] = station_loc
        sdf['wind_speed_units'] = wsu

    else:
        print(f'Unexpected result - found no observations for station')
    sdf.set_index='obs_datetime'
    return(sdf)

In [44]:
def on_build():
    try:
        response = requests.get(url)
        station_count = len(response.json()['STATION'])
        print(f'Found data from {station_count} stations')
        # to do: add progress indicator here
        list_of_dicts = []
        for i in range(0,station_count):
            sdf = add_station_data(response, i)
            list_of_dicts.append(sdf.to_dict())

        df = pd.DataFrame(list_of_dicts)
        return(df)
    except:
        print("Problem getting weather data from API")
        print(response.text)
        return(None)


In [45]:
interact_manual(on_build,i=build_button);

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

#### Extra leftovers

In [ ]:
sdf = add_station_data(response, 1)
sdf.head()

In [ ]:
#df = pd.DataFrame(columns=['station_id','station_name','obs_datetime','wind_speed','wind_speed_units','wind_direction','wkt'])

list_of_dicts = []
for i in range(0,station_count):
    sdf = add_station_data(response, i)
    print(sdf.shape)
    print(sdf.columns)
    list_of_dicts.append(sdf.to_dict())

df = pd.DataFrame(list_of_dicts)

In [111]:
tesselo_url = 'https://pixels.tesselo.com/tiles/{z}/{x}/{y}.png?'
tesselo_url += 'end=2018-11-04&start=2018-10-21&max_cloud_cover_percentage=10'
tesselo_url

'https://pixels.tesselo.com/tiles/{z}/{x}/{y}.png?end=2018-11-04&start=2018-10-21&max_cloud_cover_percentage=10'

var tms_ne = L.tileLayer('https://demo.boundlessgeo.com/geoserver/gwc/service/tms/1.0.0/ne:ne@EPSG:900913@png/{z}/{x}/{y}.png', {
    tms: true
}).addTo(map);

In [112]:
from ipyleaflet import Map, basemaps, basemap_to_tiles

m = Map(
    layers=((tesselo_url, "Tesselo Sentinel-2"), ),
    center=(location.latitude, location.longitude),
    zoom=4
)

m

AttributeError: 'tuple' object has no attribute 'model_id'

In [24]:
# from
# https://weather.msfc.nasa.gov/goes/gislist.html

goes_layer = 'realtime_goes16_abi_conus_band02_0p64um'
goes_layer = 'realtime_goes16_abi_conus_fireTemperature'

goes_latest_wms = 'https://wms1.nsstc.nasa.gov/geoserver/GOES/wms?'
goes_latest_wms += 'SERVICE=WMS'

goes_latest_wms += f'&layers={goes_layer}'
#goes_latest_wms += '&bbox=-100,25,-75,50'
#goes_latest_wms += '&request=getmap'
#goes_latest_wms += '&width=800&height=600'
#goes_latest_wms += '&format=image/png'

# var wmsLayer = L.tileLayer.wms('https://demo.boundlessgeo.com/geoserver/ows?', wmsOptions).addTo(map);


In [18]:
from ipyleaflet import WMSLayer, Map

In [32]:
m = Map(center=(location.latitude, location.longitude), zoom=10)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [25]:
wms = WMSLayer(url='https://ows.terrestris.de/osm/service', layers='OSM-Overlay-WMS', tile_size=512)
m.add_layer(wms)

In [28]:
goes_layer = WMSLayer(url=goes_latest_wms, layers='GOES-WMS', tile_size=256)
m.add_layer(goes_layer)


In [8]:
from ipyleaflet import Map, basemaps, basemap_to_tiles

m = Map(
    layers=(basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2017-04-08"), ),
    center=(location.latitude, location.longitude),
    zoom=4
)

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

#### Store in OmniSci

In [ ]:
table_name = 'placer_co_winds'

In [ ]:
mapdql(f"DROP TABLE {table_name}")

In [ ]:
sdf.columns

In [ ]:
ddl = "station_id TEXT ENCODING DICT, station_name TEXT ENCODING DICT, "
ddl += "obs_datetime timestamp, wind_speed float, wind_speed_units TEXT ENCODING DICT, "
ddl += "wind_direction float, omnisci_geo GEOMETRY(point,4326)"

In [ ]:
mapdql(f"CREATE TABLE {table_name} ({ddl})")

In [ ]:
pwdl = !pwd
cwd = pwdl[0]
full_path = cwd + '/' + 'placer_co_winds.csv'
full_path

In [ ]:
for i in range(0,station_count):
    sdf = add_station_data(response, i)
    sdf.to_csv(full_path,index=False)
    mapdql(f"COPY {table_name} FROM '{full_path}'")